# Data Download and Exploration

This code means that the notebook will re-import your source code in `src` when it is edited (the default is not to re-import, because most modules are assumed not to change over time).  It's a good idea to include it in any exploratory notebook that uses `src` code

In [1]:
%load_ext autoreload
%autoreload 2

This snippet allows the notebook to import from the `src` module.  The directory structure looks like:

```
├── notebooks          <- Jupyter notebooks. Naming convention is a number (for ordering)
│   │                     followed by the topic of the notebook, e.g.
│   │                     01_data_collection_exploration.ipynb
│   └── exploratory    <- Raw, flow-of-consciousness, work-in-progress notebooks
│   └── report         <- Final summary notebook(s)
│
├── src                <- Source code for use in this project
│   ├── data           <- Scripts to download and query data
│   │   ├── sql        <- SQL scripts. Naming convention is a number (for ordering)
│   │   │                 followed by the topic of the script, e.g.
│   │   │                 03_create_train_table.sql
│   │   ├── data_collection.py
│   │   └── sql_utils.py
```

So we need to go up two "pardir"s (parent directories) to import the `src` code from this notebook.  You'll want to include this code at the top of any notebook that uses the `src` code.

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    

The code to download all of the data and load it into a SQL database is in the `data` module within the `src` module.  You'll only need to run `download_data_and_load_into_sql` one time for the duration of the project.

In [3]:
from src.data import data_collection

In [4]:
# Download the competition zip data from kaggle
# Note: You will need a kaggle API key in order to do this
! kaggle competitions download -c riiid-test-answer-prediction -p ../../data

This line may take as long as 10-20 minutes depending on your network connection and computer specs

In [7]:
data_collection.download_data_and_load_into_sql()

Successfully created database and all tables

Successfully loaded CSV file into `train` table
        
Successfully loaded CSV file into `questions` table
        
Successfully loaded CSV file into `lectures` table
        
Successfully loaded CSV file into `example_test` table
        


Now we can access our data from our SQL database

In [13]:
import psycopg2
import pandas as pd

In [14]:
DBNAME = "riiid_education"

In [15]:
conn = psycopg2.connect(dbname=DBNAME)

In [18]:
pd.read_sql("SELECT * FROM train LIMIT 10;", conn)

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,79079671,9.165859e+09,1683350506,5769,0,1296,0,1,35000.0,True
1,79079672,9.165927e+09,1683350506,5038,0,1297,0,1,24000.0,True
2,79079673,9.165980e+09,1683350506,9109,0,1298,3,1,37000.0,True
3,79079674,9.251019e+09,1683350506,589,0,1299,3,1,23000.0,True
4,79079675,9.251452e+09,1683350506,1161,0,1300,1,1,19000.0,True
5,79079676,9.251877e+09,1683350506,787,0,1301,0,0,20000.0,True
6,79079677,9.251916e+09,1683350506,837,0,1302,3,1,19000.0,True
7,79079678,9.251995e+09,1683350506,501,0,1303,3,1,18000.0,True
8,79079679,9.252088e+09,1683350506,731,0,1304,0,0,22000.0,True
9,79079680,9.252144e+09,1683350506,1369,0,1305,1,1,21000.0,True


Notice the `LIMIT 10` above.  These tables have a large amount of data in them and **your goal is to use SQL to create your main query, not Pandas**.  Pandas can technically do everything that you need to do, but it will be much slower and more inefficient.  Nevertheless, Pandas is still a useful tool for exploring the data and getting a basic sense of what you're looking at.

Make sure you close the DB connection when you are done using it

In [12]:
conn.close()